### NetCDF files to .hyper for use in Tableau

Recently, Klaus Schulte asked for some help with wrestling NetCDF files into Tableau.  I wrote up a little script with Python to pull the data out and convert it to a .hyper file for use in Tableau and figured that I should go ahead and document and share the script in case anyone else had a similar problem - so here it is!

This Jupyter notebook will walk through the steps of exploring and processing NetCDF files for use in Tableau.

A blog post with some more details on the Tableau side of working with the data is [here](fill in the url)

Note that this example is a simple NetCDF file, and not all are this straightforward in terms of their structure, but some resources on NetCDF files are linked below.

#### What is NetCDF?
NetCDF files are a special file format for storing array-oriented scientific data.  

Typically you'll have a grid of locations and then a number of attributes associated with each of those locations.  For each attribute at each location you might have multiple measurements or time steps.  The data gets complicated pretty fast!  

The [UCAR web site on NetCDF files](https://www.unidata.ucar.edu/software/netcdf/) has all sorts of information about the file format, structure, and use cases!  I highly recommend scrolling through if you have to create or deal with NetCDF files 

### Get to the good stuff, the script!
The first thing we need to do is import some useful libraries for working with the data.  In this case, I'm using [Pandas](https://pandas.pydata.org/) and [netCDF4](https://pypi.org/project/netCDF4/).  These are what I will use to read in a NetCDF file and manipulate the data so that it's in a good shape to put into a .hyper file using the [Tableau Hyper API](https://help.tableau.com/current/api/hyper_api/en-us/index.html) 

In [1]:
# import some useful libraries
import netCDF4
import pandas as pd

Next, let's grab some NetCDF files to work with, and then make sure that Python is going to know where to find them.

I'm going to work with files from [this site](https://www.ufz.de/index.php?de=37937)

Specifically, I'll use this monster huge file [here](https://www.ufz.de/export/data/2/269199_SMI_SM_L02_Oberboden_monatlich_1951-01_2021-12_inv.nc) 

This really is a large file, so when we get to the processing part of the script I will restrict to just one time step - I'll make a note of it later when we get to that part and will talk about how to expand out to run the whole file.

In general, it's a good idea with scripts like this to start small and check to make sure that a small bit of the data is working as expected before you run it on the whole dataset.


In [2]:
# Where is the data stored?
# this assumes the file is in the same directory as the script
INPUT_NETCDF = r'269200_SMI_SM_Lall_Gesamtboden_monatlich_1951-01_2021-12_inv.nc'

# While we're up here at the start of the script, we might as well set up the info for our output .hyper file too
# this will put the output file in the same directory with the input file
OUTPUT_HYPER = INPUT_NETCDF[:-3] + '.hyper' # use the name of the input .nc file, but it'll be a .hyper at the end

#### Read in the data!
We can use the netCDF4 library to easily read in the data and to explore how it's structured and what the general contents are.

Here is a web page that I found helpful for figuring out the netCDF4 library
https://unidata.github.io/netcdf4-python/

In [3]:
# read in the file to a variable named 'nc' 
nc = netCDF4.Dataset(INPUT_NETCDF)

Now that we have the file read in, we can explore the contents.  We can start by just looking at the basic header info - I'll summarize it here, and the next cell in this notebook we will ask for the details to be printed out so that you can see what the info looks like for the file

That tells us details on the dimensions in the data:
* Time (size: 14)
* easting (size: 175)
* northing (size: 225)

What does this tell us? 
There are 14 time steps stored in the NetCDF
The data is in a 225x175 grid

What are the variables?
* Time (time)
* Lon (northing, easting)
* Lat (northing, easting)
* Easting (easting)
* Northing (northing)
* SMI (time, northing, easting)

In [4]:
# print out the details for the netcdf file
print(nc)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    NCO: netCDF Operators version 4.9.8 (Homepage = http://nco.sf.net, Code = http://github.com/nco/nco)
    Conventions: CF-1.8
    institution: Helmholtz Center for Environmental Research
    originator: UFZ Drought Monitor
    contact: klima@ufz.de
    crs: EPSG:31468
    source: UFZ Drought Monitor / Helmholtz Centre for Environmental Research
    creation_date: 2022-08-11
    dimensions(sizes): time(852), easting(175), northing(225)
    variables(dimensions): int32 time(time), float32 SMI(time, northing, easting), float64 lat(northing, easting), float64 lon(northing, easting)
    groups: 


####  Walk through the NetCDF file and grab the attributes we want

Since the data is in a big raster, we need to start with knowing how many cells we are dealing with.  

Tableau doesn't work with raster data, so we're going to want to pick up a single lat/lng value for each cell and data for whatever variables / time steps are interesting to us in the NetCDF.

Let's start with the dimensions for the grid, so we can iterate through and grab the values to drop into our tables in .hyper

The goal is to end up with a big file that stores the location, the attribute, and the time step.  It'll be a long list with one row for each cell & time step combination.


In [5]:
# let's get the dimensions for width and height of the grid of data 
y_size = nc.dimensions['northing'].size # northing coordinate count
x_size = nc.dimensions['easting'].size # easting coordinate count
print(y_size, x_size)

225 175


In [6]:
# set up two lists for holding our data
# this will make it easy to shovel them into a .hyper file at the end
data = [] # this will store all of the data values

# this will store the geometry.  Since a NetCDF file is multi-dimensional 
# (in this case we have a number of time steps) we don't need to repeat the geometry for each time step...
# we just need to store it once and have a key to link it to the attributes
data_geom = [] 

Now the time consuming part - we're going to walk through the NetCDF file and look at every cell and store the values of interest

In this case I'm really making two lists of data:
* The Geometry - I don't need to duplicate this for every record, since it would be exactly the same for each time step.  So I just write an ID field and the latitude and longitude and store that to put in a table in the .hyper.  Then I can join or relate the Geometry to the attributes in Tableau.

* The data - I will end up with one row in my data table for each cell in the NetCDF raster.  In that row, I'll have an ID field (so I can match it with the geometry table), the time step, and the data value(s) of interest

To collect all of this, we just walk through the tables.  In this file, I've simplified to just write the data for 1 time step.  If you want to get the data for ALL of the time steps, you just change the end value for the range in the first FOR statement below (see the comment)

In [7]:
print('Currently collecting data for this time step...')

# grabbing 1 time step for a test - use ** len(times) ** in the for loop instead of hard coding the number "1" 
# as the max value in the loop if you want to go through all of the data instead...
for i in range(0, 1): 
    print(i) # just printing out some info so you know what time step is being processed...delete it if it's too chatty for you
    
    # walk through every cell in the grid (go through each cell one at a time...just walk along the rows and columns)
    for x in range(0, x_size):
        for y in range(0, y_size):
            # make one list with just id, lat, lng (for the geom table in .hyper)
            # we only need to do this once, so we'll just grab the data when we go through the first time step
            if i == 0:
                data_geom.append([f'{y}-{x}',  # id string for the point location
                                  nc.variables['lat'][y][x], # grab the latitude for that row/column 
                                  nc.variables['lon'][y][x]]) # grab the longitude for that row/column

            # don't bother with the masked values (no data value recorded there, it's just an empty cell in the grid)
            if nc.variables['SMI'][i].mask[y][x] == False:
                data.append([f'{y}-{x}',  #id string for the point location
                             nc.variables['time'][i], # the time step ID
                             nc.variables['SMI'][i][y][x]]) # the value for that location & time

Currently collecting data for this time step...
0


Whew...now that is all done, we can just drop everything into a .hyper file for use in Tableau.

If you want details, and sample code, to explain how the Hyper API works, check out the [great documentation](https://help.tableau.com/current/api/hyper_api/en-us/index.html)


In [8]:
# Make the .hyper file
from tableauhyperapi import HyperProcess, Telemetry, SqlType, TableDefinition, Connection, CreateMode,TableName, Inserter

with HyperProcess(telemetry=Telemetry.SEND_USAGE_DATA_TO_TABLEAU) as hyper:
        with Connection(endpoint=hyper.endpoint,
                        create_mode=CreateMode.CREATE_AND_REPLACE,
                        database=OUTPUT_HYPER) as connection:
            
            print('Output database is open...')

            # Create the schemas
            connection.catalog.create_schema('Data')
            connection.catalog.create_schema('Geom')

            # Create the table definitions
            schema_data = TableDefinition(table_name=TableName('Data', 'Data'),
                                     columns=[
                                         TableDefinition.Column('ID', SqlType.text()),
                                         TableDefinition.Column('Time', SqlType.double()),
                                         TableDefinition.Column('Value', SqlType.double()),
                                     ])

            schema_geom = TableDefinition(table_name=TableName('Geom', 'Geom'),
                                     columns=[
                                         TableDefinition.Column('ID', SqlType.text()),
                                         TableDefinition.Column('Latitude', SqlType.double()),
                                         TableDefinition.Column('Longitude', SqlType.double())
                                     ])

            # Create the tables in the connection catalog
            connection.catalog.create_table(schema_data)
            connection.catalog.create_table(schema_geom)

            # Insert the data from lists
            with Inserter(connection, schema_data) as inserter:
                inserter.add_rows(data)
                inserter.execute()
            with Inserter(connection, schema_geom) as inserter:
                inserter.add_rows(data_geom)
                inserter.execute()
        
        print('Connection to write new .hyper is closed')

Output database is open...
Connection to write new .hyper is closed
